In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import matplotlib.backends.backend_pdf

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

%matplotlib inline

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/day/'

In [ ]:
yrs=np.arange(1979,2021,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir2+'mswep_past_deres_'+str(year)+'.nc')
    #precip=ds['precip'].sel(lon=np.arange(-130,0.25,0.25),lat=np.arange(15,65.25,0.25))[:,0::4,0::4]
    precip=ds['precip'][:,0::4,0::4]
    
    if i==0:
        precip_concat=precip
    else:
        precip_concat=xr.concat([precip_concat,precip],dim='time')

In [ ]:
y_indices=np.random.choice(np.arange(precip_concat.lat.size),10)
x_indices=np.random.choice(np.arange(precip_concat.lon.size),10)

#lat_index=int(precip_concat.lat.size/2)
#lon_index=int(precip_concat.lon.size/2)
mean_distance_list=[]
for x in x_indices:
    for y in y_indices:
        lat_index=y
        lon_index=x

        correlations=xr.zeros_like(precip_concat[0,:,:])
        time_series_0=precip_concat[:,lat_index,lon_index].values.tolist()
        for j in range(precip_concat.lat.size):
            for k in range(precip_concat.lon.size):
                time_series=precip_concat[:,j,k].values.tolist()
                corr=np.corrcoef(time_series,time_series_0)[0,1]
                correlations[j,k]=corr
        print('A')
        fig=plt.figure(figsize=(20,7))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        cax=ax.pcolormesh(correlations.lon,correlations.lat,correlations,vmin=-1,vmax=1,cmap=plt.cm.seismic)
        cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)
        ax.coastlines(resolution='10m')
        c=ax.contour(correlations.lon,correlations.lat,correlations,levels=[1./2.718],colors='k',linewidths=2.0)
        
        lat_0=correlations.lat.values[lat_index]
        lon_0=correlations.lon.values[lon_index]
        #print((lon_0,lat_0))
        vertices=c.allsegs
        #print(vertices)
        distance_list=[]
        for i in range(len(vertices)):
            lon=vertices[0][0][i][0]
            lat=vertices[0][0][i][1]
            distance=geopy.distance.distance((lat_0,lon_0),(lat,lon)).km
            distance_list.append(distance)
        mean_distance=float(sum(distance_list))/float(len(distance_list))
        print(mean_distance)
        
        ax.set_title('Correlation e-folding distance',fontsize=32,pad=5)
        ax.plot(correlations.lon.values[lon_index],correlations.lat.values[lat_index],'ko',markersize=10)
        g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
        cbar.ax.tick_params(labelsize=21) 
        cbar.set_label('% yr$^{-1}$',fontsize=23,labelpad=5)
        ax.coastlines(resolution='10m')
        countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
        ax.add_feature(countries)
        g1.xlabel_style={'size':21,'color':'k'}
        g1.ylabel_style={'size':21,'color':'k'}
        g1.xformatter=LONGITUDE_FORMATTER
        g1.yformatter=LATITUDE_FORMATTER
        g1.top_labels=False
        g1.right_labels=False

        #ax.text(-128,61,'Mean = '+str(round(mean_distance,2)),fontsize=18)
        plt.show()
        
        mean_distance_list.append(mean_distance)

In [ ]:
fig=plt.figure(figsize=(20,7))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
cax=ax.pcolormesh(correlations.lon,correlations.lat,correlations,vmin=-1,vmax=1,cmap=plt.cm.seismic)
cbar=plt.colorbar(cax,pad=0,fraction=0.046,aspect=25)
ax.coastlines(resolution='10m')
c=ax.contour(correlations.lon,correlations.lat,correlations,levels=[1./2.718],colors='k',linewidths=2.0)
ax.set_title('Correlation e-folding distance',fontsize=32,pad=5)
ax.plot(correlations.lon.values[lon_index],correlations.lat.values[lat_index],'ko',markersize=10)
g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,linewidth=1.5,color='gray',alpha=0.0,linestyle='--')
cbar.ax.tick_params(labelsize=21) 
cbar.set_label('% yr$^{-1}$',fontsize=23,labelpad=5)
ax.coastlines(resolution='10m')
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
g1.xlabel_style={'size':21,'color':'k'}
g1.ylabel_style={'size':21,'color':'k'}
g1.xformatter=LONGITUDE_FORMATTER
g1.yformatter=LATITUDE_FORMATTER
g1.top_labels=False
g1.right_labels=False

ax.text(-128,61,'Mean = '+str(round(mean_distance,2)),fontsize=18)
plt.show()

In [ ]:
fig.savefig(dir_data+'precip_correlation_1.png')